In [1]:
import numpy as np
from numpy.linalg import norm
from sklearn.neighbors import NearestNeighbors

import os

import tensorflow as tf
from tensorflow.keras import applications
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.applications.resnet50 import ResNet50
# import tensorflow.keras.applications.resnet50 as resnet50

In [2]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

In [3]:
def extract_features(img_path, model, preprocess_input):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path,
                        target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    features = model.predict(preprocessed_img)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features)
    return normalized_features

In [4]:
test_filenames = sorted(get_file_list('./cat_test_images'))
test_filenames

['./cat_test_images\\jjokgo1.jpg',
 './cat_test_images\\jjokgo2.jpg',
 './cat_test_images\\jjokgo3.jpg',
 './cat_test_images\\jjokgo4.jpg',
 './cat_test_images\\jjokgo5.jpg',
 './cat_test_images\\mango1.jpg',
 './cat_test_images\\mango2.jpg',
 './cat_test_images\\mango3.jpg',
 './cat_test_images\\mango4.jpg',
 './cat_test_images\\mango5.jpg',
 './cat_test_images\\sabum1.jpg',
 './cat_test_images\\sabum2.jpg',
 './cat_test_images\\sabum3.jpg',
 './cat_test_images\\sabum4.jpg',
 './cat_test_images\\sabum5.jpg',
 './cat_test_images\\samsak1.jpg',
 './cat_test_images\\samsak2.jpg',
 './cat_test_images\\samsak3.jpg',
 './cat_test_images\\samsak4.jpg',
 './cat_test_images\\samsak5.jpg',
 './cat_test_images\\sango1.jpg',
 './cat_test_images\\sango2.jpg',
 './cat_test_images\\sango3.jpg',
 './cat_test_images\\sango4.jpg',
 './cat_test_images\\sango5.jpg',
 './cat_test_images\\yuksi1.jpg',
 './cat_test_images\\yuksi2.jpg',
 './cat_test_images\\yuksi3.jpg',
 './cat_test_images\\yuksi4.jpg',
 './

In [5]:
import re

def catname(idx):
    return re.sub(r"[0-9]", '', test_filenames[idx].split("\\")[1].split(".")[0])

In [6]:
def scoring(neighbors, distances, n):
    result_score = 0
    
    for test_num in range(len(test_filenames)):
        cur_score = n
        cur_cat = catname(test_num)
        prediction = dict()
#         print(cur_cat)
        
        for item in neighbors.kneighbors([distances[test_num]], return_distance=False)[0]:
            if cur_score == n:
                cur_score -= 1
                continue
            
            if catname(item) in prediction:
                prediction[catname(item)] += cur_score
#                 print(catname(item) + " " + str(cur_score))
            else:
                prediction[catname(item)] = cur_score
#                 print(catname(item) + " " + str(cur_score))
                
            cur_score -= 1
        
        rank = sorted(prediction.items(), key=(lambda x: x[1]), reverse=True)
#         print(rank)
        
        if rank[0][0] == cur_cat:
            result_score += 2
        elif rank[1][0] == cur_cat:
            result_score += 1
    
    print(str(result_score) + " / " + str(len(test_filenames) * 2))
    return result_score

In [7]:
def test_model(model, preprocess_input, n_neighbors):
    result =  []

    for filename in test_filenames:
        result.append(extract_features(filename, model, preprocess_input))

    neighbors = NearestNeighbors(n_neighbors=n_neighbors + 1,
                                algorithm='brute',
                                metric='euclidean').fit(result)

    # minkowski, euclidean, mahalanobis

    scoring(neighbors, result, n_neighbors + 1)

In [28]:
n = 5

resnet50_model = applications.resnet50.ResNet50(weights='imagenet',
                include_top=True,
                input_shape=(224, 224, 3))

model_epoch1 = tf.keras.models.load_model("./model-finetuned-210102.h5")

vgg16_model = applications.vgg16.VGG16(weights='imagenet',
                include_top=True,
                input_shape=(224, 224, 3))

mobilenet_model = applications.mobilenet.MobileNet(weights='imagenet',
                include_top=True,
                input_shape=(224, 224, 3))

densenet_model = applications.densenet.DenseNet201(weights='imagenet',
                include_top=True,
                input_shape=(224, 224, 3))

nasnet_model = applications.nasnet.NASNetMobile(weights='imagenet',
                include_top=True,
                input_shape=(224, 224, 3))

mobilenet_v2_model = applications.mobilenet_v2.MobileNetV2(weights='imagenet',
                include_top=True,
                input_shape=(224, 224, 3))

14540800/14536120 [==============================] - 1s 0us/step


In [21]:
print("ResNet50")
test_model(resnet50_model, applications.resnet50.preprocess_input, n)

ResNet50
23 / 70


In [18]:
print("trained 1 epoch from ResNet50")
test_model(model_epoch1, applications.resnet50.preprocess_input, n)

trained 1 epoch from ResNet50
46 / 70


In [19]:
print("VGG16")
test_model(vgg16_model, applications.vgg16.preprocess_input, n)

VGG16
23 / 70


In [20]:
print("MobileNet")
test_model(mobilenet_model, applications.mobilenet.preprocess_input, n)

MobileNet
19 / 70


In [24]:
print("DenseNet201")
test_model(densenet_model, applications.densenet.preprocess_input, n)

DenseNet201
36 / 70


In [27]:
print("NASNet Mobile")
test_model(nasnet_model, applications.nasnet.preprocess_input, n)

NASNet Mobile
13 / 70


In [29]:
print("MobileNet V2")
test_model(mobilenet_v2_model, applications.mobilenet_v2.preprocess_input, n)

MobileNet V2
22 / 70
